In [2]:
import os
import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"
import plotly.io as pio
pio.renderers.default = "jupyterlab"
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import analysis.data_importing as imp  # Custom importing module
import analysis.plotting as pl  # Custom plotting module
import interfaces.postprocessing as pif  # post processing interface
import scipy.signal
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)


def cross_correlate(series1, series2):
    sig1 = series1.dropna()
    sig2 = series2.dropna()
    corr = scipy.signal.correlate(sig1, sig2)
    lags = scipy.signal.correlation_lags(len(sig1), len(sig2))

    return corr / corr.max(), lags


def plot_cross_corr(series1, series2, corr, lags, title="", show=False, renderer = ""):
    fig = make_subplots(rows=2, cols=1)
    fig.update_layout(title=title)
    fig.update_xaxes(title = "Time [s]", row=1, col=1)
    fig.update_yaxes(title = "VT", row=1, col=1)
    fig.update_xaxes(title = "Lag [s]", row=2, col=1)
    fig.update_yaxes(title = "Cross Correlation", row=2, col=1)
    fig = fig.add_trace(go.Scatter(
        x=series1.index,
        y=series1.values,
        name="Series1"
    ), row=1, col=1)
    fig = fig.add_trace(go.Scatter(
        x=series2.index,
        y=series2.values,
        name="Series2"
    ), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=lags,
        y=corr,
        name="Cross-Correlation"
    ), row=2, col=1)
    if show:
        fig.show()
    return fig

In [13]:
# Get and clean data
activity_data_dir = "data/Arnar_Larusson_2023-04-12_easy_ride"
uncleaned_data_dir = os.path.join(activity_data_dir, "uncleaned_data")
cleaned_data_dir = os.path.join(activity_data_dir, "cleaned_data")
imp.clean_all_data(uncleaned_data_dir)
clean_dfs = imp.load_cleaned_data(cleaned_data_dir)

raw_slow_df = clean_dfs["raw_slow_df"]
aws_b3_df = clean_dfs["aws_b3_df"]
live_b3_df = clean_dfs["live_b3_df"]
raw_fast_df = clean_dfs["raw_fast_df"]

# Post Processing Data from BR_rVE_RTformat
chest_raw, chest_5hz, chest_bs, chest_bs_smooth, time, X_bbyb_df = pif.BR_rVE_RTformat_wrapper(uncleaned_data_dir)


cleaned_data folder already exists at data/Arnar_Larusson_2023-04-12_easy_ride\cleaned_data


In [5]:
X_bbyb_df

,breath-by-breath Time,BR Inst,BR ORA,TV Inst,TV Smoothed,VE Inst,VE Smoothed,In/Ex Ratio Inst,In/Ex Ratio Smoothed
0,5.118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9.558,13.51,NaN,36.5,141.5,4.9,27.3,0.500,2.041
2,13.398,15.62,NaN,50.4,142.7,7.9,27.0,4.176,1.949
3,16.920,17.05,NaN,622.9,143.9,106.2,26.7,2.769,1.856
4,18.882,30.61,NaN,155.5,145.1,47.6,26.3,0.272,1.764
...,...,...,...,...,...,...,...,...,...
823,2558.238,31.91,28.4,245.1,226.1,78.2,48.0,0.757,1.097
824,2560.842,23.08,28.4,228.6,234.4,52.7,49.5,0.969,1.098
825,2563.362,23.81,27.0,178.4,242.7,42.5,51.0,0.656,1.099
826,2565.480,28.30,26.5,130.1,251.0,36.8,52.4,2.852,1.099


# Get and clean data
VT_p :  post processing VT data
VT_pi : post processing VT data converted to integer values
VT_l : live VT data

In [ ]:
VT_p = pd.DataFrame(aws_b3_df["VT"].set_axis(aws_b3_df["breathTime"]))
VT_pi = pd.DataFrame(aws_b3_df["VT"].set_axis(aws_b3_df["breathTime"].astype(int)))
VT_l = pd.DataFrame(live_b3_df["VT"].set_axis(live_b3_df["breathTime"].astype(int)))

In [ ]:
# View data as table
VT_view = VT_p.join(VT_pi, how="outer", lsuffix="_p", rsuffix="_pi").join(VT_l, how="outer", rsuffix="_l").rename(columns={"VT" : "VT_l"})
VT_view

# Plot Original VT Data
The original VT data is plotted below. There is a negligible difference between the VT_p and VT_pi data.

In [ ]:
plot_dict1 = {
    "Post Processing (Original)" : (VT_p, "index" ,"VT"),
    "Post Processing (Int)" : (VT_pi, "index","VT"),
    "Live" : (VT_l, "index", "VT")
}
pl.plot_df_columns(plot_dict1, plottitle = "Original Signals", xtitle="Time [s]", ytitle= "VT", show = False, renderer = "jupyterlab")


# Cross Correlation
Cross-correlation is used to find the best offset between the two signals.
It is defined as:
$$
\begin{align}
C_{xy}[k] = \sum_{n=0}^{N-1} x[n]y[n-k]
\end{align}
$$
where $x[\cdot]$ and $y[\cdot]$ are the two signals and $k$ is the offset.

The offset is found by finding the lag $k$ that maximizes value of the cross-correlation.

However, our data is non-uniformly sampled.  Each measurement corresponds to a breath-detection in their corresponding algorithm. First, we must interpolate the data to get each series on the same time-scale

# Interpolate Data

In [ ]:
# Create new dataframe by joining the two dataframes
VT_j = VT_pi.join(VT_l, how="outer", lsuffix="_pi", rsuffix="_l")
VT_j

In [ ]:
# Interpolate the data based on the index
VT_j_int = VT_j.interpolate(method="index").fillna(value = 0) #fillna needed due to mismatch in length of each VT series
VT_j_int

In [ ]:
# Plot Cross Correlation of interpolated series
corr, lags = cross_correlate(VT_j_int["VT_pi"], VT_j_int["VT_l"])
plot_cross_corr(VT_j_int["VT_pi"], VT_j_int["VT_l"], corr, lags, title="Cross Correlation of Interpolated Data", show=False)

Note: The VT metrics in VT_j_int are interpolated across the union of the breath times between the (integer) breath times of the post processing algorithm and the breath times of the live algorithm.  This is done to ensure that the two series are on the same time scale.  The interpolated values are filled with 0 to ensure that the cross-correlation is not biased by the length of the series.

## Resampling to second-by-second intervals from breath-by-breath intervals



In [ ]:
# Create time df with
d = pd.DataFrame(np.arange(max(VT_j_int.index)), index = np.arange(max(VT_j_int.index)))
VT_sec = VT_j_int.join(d, how="outer")
# VT_j_int_sec # Uncomment to view
VT_sec_int = VT_sec.interpolate(method="index").fillna(value = 0).drop(columns=[0])
VT_sec_int


In [ ]:
# Plot Cross Correlation of interpolated series
corr, lags = cross_correlate(VT_sec_int["VT_pi"], VT_sec_int["VT_l"])
plot_cross_corr(VT_sec_int["VT_pi"], VT_sec_int["VT_l"], corr, lags, title="Cross Correlation of Interpolated Data", show=False)

Resampling to second-by-second intervals and interpolating actually gave us a new peak on the cross correlation.  Now we shift by this lag to align the two series.


In [ ]:
# Shift the data by the argmax_lag of the cross correlation data
opt_lag = lags[np.argmax(corr)]
VT_sec_int["VT_l_shift"] = VT_sec_int["VT_l"].shift(opt_lag)
VT_sec_int

In [ ]:
# Plot Cross Correlation of shifted series
corr, lags = cross_correlate(VT_sec_int["VT_pi"], VT_sec_int["VT_l_shift"])
plot_cross_corr(VT_sec_int["VT_pi"], VT_sec_int["VT_l_shift"], corr, lags, title="Cross Correlation of Interpolated Data", show=False)

Now the peak of the cross correlation is at 0 lag.  This means that the two series have the best alignment based on shifting the entirety of the live data by a constant lag


In [ ]:
# Compute the error between the two VT series
VT_sec_int["error"] = VT_sec_int["VT_pi"] - VT_sec_int["VT_l_shift"]
VT_sec_int

In [ ]:
# Plot the error
plot_dict2 = {
    "VT_pi": (VT_sec_int, "index", "VT_pi"),
    "VT_l_shift": (VT_sec_int, "index", "VT_l_shift"),
    "Error" : (VT_sec_int, "index", "error")
}
fig = pl.plot_df_columns(plot_dict2, plottitle = "Error", xtitle="Time [s]", ytitle= "Error", show = False, renderer = "jupyterlab")
max_error = max(VT_sec_int["error"].max(), abs(VT_sec_int["error"].min()))
fig.update_yaxes(range=[-max_error*1.1, max_error*1.1])
# Color the figs traces such that if the error is negative its blue and positive it is red


In [ ]:
# Sum the absolute value of the error as a function of time
VT_sec_int["abs_error"] = VT_sec_int["error"].abs()
# Perform a cumulative sum of the absolute error from time 0 to time t
VT_sec_int["cum_abs_error"] = VT_sec_int["abs_error"].cumsum()
VT_sec_int
# Average the cumulative error over time
VT_sec_int["avg_cum_abs_error"] = VT_sec_int["cum_abs_error"] / VT_sec_int.index
VT_sec_int


In [ ]:
# Get a running average of the error
VT_sec_int["running_avg_error"] = VT_sec_int["error"].rolling(window=60).mean()
# Plot the running average of the error with the VT data
plot_dict_ra = {
    "Running Average Error" : (VT_sec_int, "index", "running_avg_error")}
fig = pl.create_subplots([plot_dict2, plot_dict_ra], plottitle = "Running Average Error", xtitles=["Time [s]","Time [s]"], ytitles= ["VT and Error", "Minute Averaged Error"], show = False)
#fig

In [ ]:
# Plot the time-averaged cumulative error with the VT data
plot_dict3 = {
    "VT_pi": (VT_sec_int, "index", "VT_pi"),
    "VT_l": (VT_sec_int, "index", "VT_l"),
    "Average Cumulative Error" : (VT_sec_int, "index", "avg_cum_abs_error")}
fig = pl.plot_df_columns(plot_dict3, plottitle = "Time-Averaged Cumulative Error", xtitle="Time [s]", ytitle= "Error", show = False, renderer = "jupyterlab")

In [ ]:
gyr = raw_fast_df[["gx", "gy", "gz"]].set_index(raw_fast_df["time"])
plot_dict_gyr = {
    "gx" : (gyr, "index", "gx"),
    "gy" : (gyr, "index", "gy"),
    "gz" : (gyr, "index", "gz")
}
fig = pl.plot_df_columns(plot_dict_gyr, plottitle = "Gyroscope Data", xtitle="Time [s]", ytitle= "Gyroscope [deg/s]", show = False, renderer = "jupyterlab")
# fig

In [ ]:
acc = raw_fast_df[["ax", "ay", "az"]].set_index(raw_fast_df["time"])
plot_dict_acc = {
    "ax" : (acc, "index", "ax"),
    "ay" : (acc, "index", "ay"),
    "az" : (acc, "index", "az")
}
fig = pl.plot_df_columns(plot_dict_acc, plottitle = "Accelerometer Data", xtitle="Time [s]", ytitle= "Accelerometer [g]", show = False, renderer = "jupyterlab")
#fig

In [ ]:
# Get motion artifact data
raw_pl = pd.DataFrame(raw_fast_df["pl"]).set_index(raw_fast_df["time"])
#raw_pl


In [ ]:
# Plot raw pl data with the previous plot data
plot_dict_pl = {
    "Raw PL" : (raw_pl, "index", "pl")
}
fig = pl.plot_df_columns(plot_dict_pl, plottitle = "Raw PL Data", xtitle="Time [s]", ytitle= "PL", show = False, renderer = "jupyterlab")
#fig

In [ ]:
import importlib
import analysis.plotting as pl
importlib.reload(pl)
# Plot VT and Raw data in subplots
raw_chest = pd.DataFrame(raw_slow_df["c"]).set_index(raw_slow_df["time"])
plot_dict_chest = {
    "Raw Chest" : (raw_chest, "index", "c")
}

df_dicts = [plot_dict_chest, plot_dict2,plot_dict_ra, plot_dict_gyr, plot_dict_acc, plot_dict_pl]
fig = pl.create_subplots(df_dicts, plottitle = "VT, Error, and Movement", xtitles= ["Time [s]","Time [s]", "Time [s]","Time [s]","Time [s]","Time [s]","Time [s]"], ytitles= ["Chest", "VT", "Average Error", "Gyroscope", "Accelerometer", "PL"], show = True)

# Minute Volume Data

In [ ]:
VE_p = pd.DataFrame(aws_b3_df["VE"]).set_axis(aws_b3_df["breathTime"])
VE_pi = pd.DataFrame(aws_b3_df["VE"]).set_axis(aws_b3_df["breathTime"].astype(int))
VE_l = pd.DataFrame(live_b3_df["VE"]).set_axis(live_b3_df["breathTime"])
# For viewing
VE_view = VE_p.join(VE_pi, how="outer", lsuffix="_p", rsuffix="_pi").join(VE_l, how="outer", rsuffix="_l")
VE_view


In [ ]:
# Plot VE data
plot_dict5 = {
    "VE_p": (VE_p, "index", "VE"),
    "VE_pi": (VE_pi, "index", "VE"),
    "VE_l": (VE_l, "index", "VE")
}
pl.plot_df_columns(plot_dict5, plottitle = "VE Data", xtitle="Time [s]", ytitle= "VE", show = False, renderer = "jupyterlab")

In [ ]:
# Perform same interpolation steps as done with VT
VE_j = VE_pi.join(VE_l, how="outer", rsuffix="_l", lsuffix="_pi")
VE_j_int = VE_j.interpolate(method="index")

In [ ]:
d = pd.DataFrame(np.arange(max(VE_j_int.index)), index = np.arange(max(VE_j_int.index)))
VE_sec = VE_j_int.join(d, how="outer")
# VT_j_int_sec # Uncomment to view
VE_sec_int = VE_sec.interpolate(method="index").fillna(value = 0).drop(columns=[0])


In [ ]:
# Compute the cross correlation of the interpolated data
corr, lags = cross_correlate(VE_sec_int["VE_pi"], VE_sec_int["VE_l"])
plot_cross_corr(VE_sec_int["VE_pi"], VE_sec_int["VE_l"], corr, lags, title="Cross Correlation of Interpolated VE Data", show=False)

In [ ]:
# Shift VE_l by the lag found in the cross correlation of VT
VE_sec_int["VE_l_shift"] = VE_sec_int["VE_l"].shift(opt_lag)
# Plot the shifted VE data with the interpolated VE data
plot_dict6 = {
    "VE_pi": (VE_sec_int, "index", "VE_pi"),
    "VE_l_shift": (VE_sec_int, "index", "VE_l_shift")
}
pl.plot_df_columns(plot_dict6, plottitle = "Shifted VE Data", xtitle="Time [s]", ytitle= "VE", show = False, renderer = "jupyterlab")

# Breathing Rate

In [ ]:
RR_p = pd.DataFrame(aws_b3_df["RRAvg"]).set_axis(aws_b3_df["breathTime"])
RR_pi = pd.DataFrame(aws_b3_df["RRAvg"]).set_axis(aws_b3_df["breathTime"].astype(int))
RR_l = pd.DataFrame(live_b3_df["RRAvg"]).set_axis(live_b3_df["breathTime"])

In [ ]:
# Plot RR data
plot_dict_RR = {
    "RR_p": (RR_p, "index", "RRAvg"),
    "RR_pi": (RR_pi, "index", "RRAvg"),
    "RR_l": (RR_l, "index", "RRAvg")
}
pl.plot_df_columns(plot_dict_RR, plottitle = "RR Data", xtitle="Time [s]", ytitle= "RR", show = False, renderer = "jupyterlab")